In [46]:
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
import pandas as pd
warnings.filterwarnings("ignore", category=FutureWarning)

#Etape 1 - Chargement des données
print('Etape 1 - Chargement des données...')
df = pd.read_csv("titanic_train.csv")

#Etape 2 - Remplacer les valeurs manquants par leurs moyennes
print('Etape 2 - Remplacer les valeurs manquants par leurs moyennes...')
df = df.fillna(df.mean())

#Etape 3 - Transformer les données qualitatives en données quantitatives
print('Etape 3 - Transformer les données qualitatives en données quantitatives...')
dict_label_encode={}
for col in df.columns:
    if df[col].dtype==object:
        dict_label_encode[col]=LabelEncoder()
        df[col]=dict_label_encode[col].fit_transform(df[col].astype(str))
        
# Etape 4 - Définir les variables
print('Etape 4 - Définir les variables...')
x = df.drop(['Survived'], axis=1)
y = df['Survived']

# Etape 5 - Séparation en df de test et validation automatique
print('Etape 5 - Séparation en df de test et validation automatique...')
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)

# Etape 6 - Définir le nombre de validation croisée - Défaut = 4
print('Etape 6 - Définir le nombre de validation croisée...')
nb_cv = input("Merci d'indiquer le nombre de validation croisée (défaut = 4) :")
nb_cv=int(nb_cv)

# Etape 7 - Contitution du Pipeline (PCA : Analyse en composante Principale / SVC : Machine à vecteurs de support)
print('Etape 7 - Contitution du Pipeline...')

# Construction d'un pipeline basé sur plusieurs approches
mon_pipeline=make_pipeline(PCA(),SVC(probability=True))

# Etape 8 - Construction du dictionnaire des paramètres
print('Etape 8 - Construction du dictionnaire des paramètres...')
param_grid = dict(pca__n_components=[5,10,x_train.shape[1]],
                  svc__C=[1,10,100,1000],
                  svc__kernel=['sigmoid','rbf'],
                  svc__gamma=[0.001,0.0001])

# Etape 9 - Construction de l'objet GridSearch et on estime les hypers-paramètres par validation croisée
print('Etape 9 - Construction de l objet GridSearch...')
grid_search_mon_pipeline = GridSearchCV(mon_pipeline, param_grid=param_grid,scoring="roc_auc",cv = nb_cv)
grid_search_mon_pipeline.fit(x_train,y_train)

# Etape 10 - La meilleur combinaison de paramètres est la suivante
print('=========================================================================================================================================')
print('La meilleure combinaison de paramètres est la suivante :', grid_search_mon_pipeline.best_params_)
print('=========================================================================================================================================')

# Etape 11 - AUC
print('Etape 11 - Calcul de l AUC...')
print('=========================================================================================================================================')
print('La valeur de l AUC optimale pour le pipeline est :', roc_auc_score(y_test,grid_search_mon_pipeline.predict_proba(x_test)[:,1]))
print('=========================================================================================================================================')

#Etape 12 - Meilleur estimateurs
print('=========================================================================================================================================')
print('Le Meilleur estimateur :', grid_search_mon_pipeline.best_estimator_)
print('=========================================================================================================================================')

# Prédire les valeurs en utilisant le dataset de TEST :
final_model= grid_search_mon_pipeline.best_estimator_
final_predictions = final_model.predict(x_test)

# Générer les résultats
Résultats = pd.DataFrame({'PassengerId': x_test.PassengerId, 'Survived': final_predictions})
Résultats

# Sauvegarder les résultats
#Résultats.to_csv('submission.csv', index=False)

Etape 1 - Chargement des données...
Etape 2 - Remplacer les valeurs manquants par leurs moyennes...
Etape 3 - Transformer les données qualitatives en données quantitatives...
Etape 4 - Définir les variables...
Etape 5 - Séparation en df de test et validation automatique...
Etape 6 - Définir le nombre de validation croisée...


Merci d'indiquer le nombre de validation croisée (défaut = 4) : 2


Etape 7 - Contitution du Pipeline...
Etape 8 - Construction du dictionnaire des paramètres...
Etape 9 - Construction de l objet GridSearch...
La meilleure combinaison de paramètres est la suivante : {'pca__n_components': 10, 'svc__C': 1, 'svc__gamma': 0.0001, 'svc__kernel': 'rbf'}
Etape 11 - Calcul de l AUC...
La valeur de l AUC optimale pour le pipeline est : 0.7163949917316325
Le Meilleur estimateur : Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=10, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('svc', SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])


,PassengerId,Survived
563,564,0
610,611,0
148,149,1
683,684,0
532,533,0
281,282,0
35,36,0
848,849,0
133,134,0
804,805,0
